In [ ]:
import os
import cv2
import glob
import numpy as np

# =========================
# Config (tune as needed)
# =========================
image_glob = "Killaloe/*.jpg"   # <-- change if needed
RATIO = 0.80                    # Lowe ratio (looser to recover weak matches)
RANSAC_REPROJ_THRESH = 4.0      # RANSAC reprojection threshold (px)
MIN_MATCH_COUNT = 8             # minimum "good" matches to attempt placement
BACK_WINDOW = 20                # how many previous frames to try as anchors
MAX_FEATURES = 8000             # SIFT nfeatures
DOWNSCALE_FOR_FEATURES = 1.0    # 1.0 = off; try 0.75 for speed once stable

# Prevent gigantic raster output (render smaller while aligning at full-res)
OUTPUT_DOWNSCALE = 0.35         # 1.0 = full-size output; 0.25–0.5 is common

# Transform sanity thresholds (per-step)
SCALE_MIN, SCALE_MAX = 0.7, 1.3
MAX_SHEAR_COS = 0.2             # |cos(angle between basis)|, 0 = orthogonal
MAX_SHIFT_FACTOR = 1.2          # max per-step shift as a fraction of image diagonal

# Transform cache file
TRANSFORM_FILE = "mosaic_transforms.npz"
USE_SAVED_TRANSFORMS = True     # If True and file exists, skip matching and just render

# =========================
# Helpers
# =========================
def load_images(pattern):
    files = sorted(glob.glob(pattern))
    if not files:
        raise IOError("No images found. Check the glob/path.")
    imgs = [cv2.imread(f) for f in files]
    ok = [(f, im) for f, im in zip(files, imgs) if im is not None]
    if not ok:
        raise IOError("Images could not be read. Check formats/paths.")
    files = [f for f, _ in ok]
    imgs  = [im for _, im in ok]
    print(f"Loaded {len(imgs)} images.")
    return files, imgs

def small(im):
    if DOWNSCALE_FOR_FEATURES == 1.0:
        return im
    h, w = im.shape[:2]
    return cv2.resize(im, (int(w*DOWNSCALE_FOR_FEATURES), int(h*DOWNSCALE_FOR_FEATURES)))

# CLAHE + light denoise helps underwater contrast
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
def prep_gray_for_features(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = clahe.apply(g)
    g = cv2.GaussianBlur(g, (3,3), 0)
    return g

def scale_pts(pts):
    """Map keypoint coords from downscaled coords back to full-res."""
    s = 1.0 / DOWNSCALE_FOR_FEATURES
    return pts * s

def transform_ok(H_2x3, img_shape):
    """
    Check similarity/affine for sane scale, shear, and shift magnitude.
    H_2x3: 2x3 affine (from estimateAffinePartial2D/2D)
    """
    h, w = img_shape[:2]
    A = H_2x3[:, :2]
    t = H_2x3[:, 2]
    # scales
    s1 = np.linalg.norm(A[:, 0]) + 1e-12
    s2 = np.linalg.norm(A[:, 1]) + 1e-12
    scale = 0.5*(s1+s2)
    if not (SCALE_MIN <= scale <= SCALE_MAX):
        return False
    # shear (basis orthogonality)
    cosang = abs(np.dot(A[:,0]/s1, A[:,1]/s2))
    if cosang > MAX_SHEAR_COS:
        return False
    # shift
    diag = np.hypot(h, w)
    if np.linalg.norm(t) > MAX_SHIFT_FACTOR*diag:
        return False
    return True

def to_homography_from_affine(A2x3):
    H = np.eye(3, dtype=np.float32)
    H[:2, :] = A2x3.astype(np.float32)
    return H

# =========================
# Compute transforms (or load cache)
# =========================
if USE_SAVED_TRANSFORMS and os.path.exists(TRANSFORM_FILE):
    data = np.load(TRANSFORM_FILE, allow_pickle=True)
    H_to_base = list(data["H_to_base"])
    sizes = list(data["sizes"])
    files = list(data["files"])
    T = data["T"]
    W = int(data["W"])
    Hh = int(data["Hh"])
    print(f"Loaded transforms from {TRANSFORM_FILE}")
else:
    # Load images once for feature extraction
    files, images = load_images(image_glob)

    # SIFT + FLANN
    try:
        sift = cv2.SIFT_create(nfeatures=MAX_FEATURES)
    except AttributeError:
        sift = cv2.xfeatures2d.SIFT_create(nfeatures=MAX_FEATURES)
    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=64))

    # Detect & describe (on downscaled/preprocessed)
    kps, descs, sizes = [], [], []
    for im in images:
        im_small = small(im)
        gs = prep_gray_for_features(im_small)
        kp, des = sift.detectAndCompute(gs, None)
        kps.append(kp)
        descs.append(des)
        sizes.append(im.shape[:2])  # full-res size

    def match_des(i, j):
        if descs[i] is None or descs[j] is None:
            return []
        matches = flann.knnMatch(descs[i], descs[j], k=2)
        good = []
        for m, n in matches:
            if m.distance < RATIO * n.distance:
                good.append(m)
        return good

    def find_H_to_anchor(i, j):
        """
        Find transform mapping image i -> image j (full-res coords).
        Prefer similarity (estimateAffinePartial2D) to avoid projective drift.
        Fallback to full affine; use homography only as a last resort.
        """
        good = match_des(i, j)
        print(f"Image {i} - {j}: {len(good)} good matches")
        if len(good) < 4:
            return None

        src = np.float32([kps[i][m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst = np.float32([kps[j][m.trainIdx].pt for m in good]).reshape(-1,1,2)
        src = scale_pts(src); dst = scale_pts(dst)

        # 1) Similarity (rotation+translation+uniform scale)
        A_sim, inl = cv2.estimateAffinePartial2D(src, dst, method=cv2.RANSAC,
                                                 ransacReprojThreshold=RANSAC_REPROJ_THRESH)
        if A_sim is not None and transform_ok(A_sim, sizes[i]):
            return to_homography_from_affine(A_sim)

        # 2) Full affine
        A_aff, inl2 = cv2.estimateAffine2D(src, dst, method=cv2.RANSAC,
                                           ransacReprojThreshold=RANSAC_REPROJ_THRESH)
        if A_aff is not None and transform_ok(A_aff, sizes[i]):
            return to_homography_from_affine(A_aff)

        # 3) Homography as last resort (but clamp projective terms)
        H, mask = cv2.findHomography(src, dst, cv2.RANSAC, RANSAC_REPROJ_THRESH)
        if H is not None and H.shape == (3,3):
            # Reject strong projective components that cause canvas blow-up
            if abs(H[2,0]) < 1e-4 and abs(H[2,1]) < 1e-4:
                A = H[:2,:2]
                s1 = np.linalg.norm(A[:,0]) + 1e-12
                s2 = np.linalg.norm(A[:,1]) + 1e-12
                scale = 0.5*(s1+s2)
                if SCALE_MIN <= scale <= SCALE_MAX:
                    return H.astype(np.float32)
        return None

    # Build accumulated transforms (to base 0)
    H_to_base = [np.eye(3, dtype=np.float32)]
    placed = [0]
    for i in range(1, len(images)):
        H_acc = None
        # dynamic expansion: try BACK_WINDOW, then 2*BACK_WINDOW
        for win in (BACK_WINDOW, 2*BACK_WINDOW):
            lookbacks = range(1, min(win, i) + 1)
            for k in lookbacks:
                j = i - k
                if H_to_base[j] is None:
                    continue
                H_ij = find_H_to_anchor(i, j)
                if H_ij is not None:
                    H_acc = (H_to_base[j] @ H_ij).astype(np.float32)
                    break
            if H_acc is not None:
                break

        if H_acc is None:
            print(f"Unable to place image {i} (no robust anchor found up to {2*BACK_WINDOW}). Skipping.")
            H_to_base.append(None)
        else:
            H_to_base.append(H_acc)
            placed.append(i)

    if not any(H is not None for H in H_to_base):
        raise RuntimeError("No images could be placed. Consider loosening thresholds or improving contrast.")

    # Compute mosaic bounds
    corners = []
    for idx, H in enumerate(H_to_base):
        if H is None:
            continue
        h, w = sizes[idx]
        pts = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
        warped = cv2.perspectiveTransform(pts, H)
        corners.append(warped)

    all_c = np.vstack(corners)
    x_min, y_min = all_c.min(axis=0).ravel()
    x_max, y_max = all_c.max(axis=0).ravel()

    W = int(np.ceil(x_max - x_min))
    Hh = int(np.ceil(y_max - y_min))
    tx = -int(np.floor(x_min)) if x_min < 0 else 0
    ty = -int(np.floor(y_min)) if y_min < 0 else 0
    T = np.array([[1,0,tx],[0,1,ty],[0,0,1]], dtype=np.float32)

    print(f"Mosaic canvas (full-res coords): {W} x {Hh}")

    # Save transforms for fast re-render later
    np.savez(TRANSFORM_FILE,
             H_to_base=np.array(H_to_base, dtype=object),
             sizes=np.array(sizes, dtype=object),
             files=np.array(files, dtype=object),
             T=T, W=W, Hh=Hh)
    print(f"Saved transforms to {TRANSFORM_FILE}")

# =========================
# Render (uses saved or freshly-computed transforms)
# =========================
S = np.array([[OUTPUT_DOWNSCALE, 0, 0],
              [0, OUTPUT_DOWNSCALE, 0],
              [0, 0, 1]], dtype=np.float32)
W_out = max(1, int(W * OUTPUT_DOWNSCALE))
H_out = max(1, int(Hh * OUTPUT_DOWNSCALE))
print(f"Rendering output canvas: {W_out} x {H_out} (scale={OUTPUT_DOWNSCALE})")

accum = np.zeros((H_out, W_out, 3), np.float32)
weight = np.zeros((H_out, W_out), np.float32)

for idx, H in enumerate(H_to_base):
    if H is None:
        continue
    # Load on demand to keep memory modest in render step
    img = cv2.imread(files[idx])
    if img is None:
        print(f"Warning: could not read {files[idx]} at render time; skipping.")
        continue
    H_off = (S @ T @ H).astype(np.float32)

    warped = cv2.warpPerspective(img, H_off, (W_out, H_out),
                                 flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    # FIXED: INTER_NEAREST (not BORDER_NEAREST)
    mask = cv2.warpPerspective(
        np.ones((sizes[idx][0], sizes[idx][1]), np.float32), H_off, (W_out, H_out),
        flags=cv2.INTER_NEAREST, borderValue=0
    )

    accum += warped.astype(np.float32) * mask[..., None]
    weight += mask

weight[weight == 0] = 1e-6
mosaic = (accum / weight[..., None]).clip(0,255).astype(np.uint8)

out_name = "stitched_mosaic.jpg"
cv2.imwrite(out_name, mosaic)
print(f"Saved {out_name}")


Loaded 59 images.
Image 1 - 0: 1991 good matches
Image 2 - 1: 1772 good matches
Image 3 - 2: 555 good matches
Image 4 - 3: 1768 good matches
Image 5 - 4: 1836 good matches
Image 6 - 5: 1484 good matches
Image 7 - 6: 800 good matches
Image 8 - 7: 170 good matches
Image 9 - 8: 772 good matches
Image 10 - 9: 1900 good matches
Image 11 - 10: 113 good matches
Image 11 - 9: 116 good matches
Image 12 - 11: 626 good matches
Image 13 - 12: 130 good matches
Image 14 - 13: 87 good matches
Image 15 - 14: 711 good matches
Image 16 - 15: 307 good matches
Image 17 - 16: 1861 good matches
Image 18 - 17: 481 good matches
Image 19 - 18: 1432 good matches
Image 20 - 19: 648 good matches
Image 21 - 20: 144 good matches
Image 22 - 21: 1562 good matches
Image 23 - 22: 1795 good matches
Image 24 - 23: 1669 good matches
Image 25 - 24: 1317 good matches
Image 26 - 25: 497 good matches
Image 27 - 26: 2799 good matches
Image 28 - 27: 2368 good matches
Image 29 - 28: 196 good matches
Image 30 - 29: 612 good match

AttributeError: module 'cv2' has no attribute 'BORDER_NEAREST'